In [407]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [408]:
pd.set_option('display.max_columns', 500)

In [409]:
#con esto veo el mes 11
mayor = "2019-12"
menor = "2019-11"

In [411]:
df = querydbtopandas(f"""
select  a11.id_item  id,

       a11.id_encuesta  id_encuesta,

       a12.id_afiliado  id_afi_afiliado,

       a12.id_prestador  id_pre_prestador,

       a17.desc_pre_nombre  desc_pre_nombre_efector,

       a17.vigente  vigente_efector,
       
       a18.desc_pre_nombre  desc_pre_nombre,
       
       a18.vigente  vigente,
       
       a18.id_pre_tipo,
       
       a12.id_prest_efector,

       a11.i_R RESPUESTA

from        DBA.ft_d_prest_encuestas        a11

       join        DBA.ft_prest_encuestas        a12

         on        (a11.id_encuesta = a12.id_encuesta)

       join        DBA.afi_afiliados        a14

         on        (a12.id_afiliado = a14.id_afiliado)

       join        DBA.d_pre_prestador        a17

         on        (a12.id_prest_efector = a17.id_pre_prestador)
         
       join        DBA.d_pre_prestador        a18

         on        (a12.id_prestador = a18.id_pre_prestador)

where (cast(fec_encuesta as date) >= date("{menor}") and cast(fec_encuesta as date) < date("{mayor}")

and a11.id_item in (1,2,3,4,5)

and a18.id_pre_tipo in ("E","C","P","I"))

group by        a11.id_item,

       a11.i_R,

       a11.id_encuesta,

       a12.id_afiliado,

       a12.id_prestador,

       a17.desc_pre_nombre,
       
       a18.id_pre_tipo,

       a17.vigente,
       
       a18.desc_pre_nombre,
       a12.id_prest_efector,
       a18.vigente""")



In [412]:
dict_id = {3: 'Atención Médica',
 2: 'Trato y Cordialidad',
 4: 'Disponibilidad de Turno',
 1: 'Puntualidad',
 5: 'Limpieza y Confort'}    

In [413]:
df["id"] = df["id"].map(dict_id)

In [414]:
#[["id_encuesta" , "fec_consumo", "id_afiliado","id_pre_prestador", "id_item","i_R", "desc_pre_nombre"]]

In [415]:
df.columns

Index(['id', 'id_encuesta', 'id_afi_afiliado', 'id_pre_prestador',
       'desc_pre_nombre_efector', 'vigente_efector', 'desc_pre_nombre',
       'vigente', 'id_pre_tipo', 'id_prest_efector', 'RESPUESTA'],
      dtype='object')

In [416]:
# cuando el tipo es e o p tomar id prestador como el 
# que realiza la practica
df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id","id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "RESPUESTA" ]]

In [417]:
# cuando el tipo de prestador es inst o circulo se toma el efector 
df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id","id_encuesta","id_afi_afiliado", "id_prest_efector", "desc_pre_nombre_efector", "RESPUESTA"]]

In [418]:
df_inst_circ.rename(columns={"id_prest_efector": "id_pre_prestador", "desc_pre_nombre_efector":"desc_pre_nombre"},inplace=True)

In [419]:
df = pd.concat([df_part_equipos, df_inst_circ])

In [420]:
df

,id,id_encuesta,id_afi_afiliado,id_pre_prestador,desc_pre_nombre,RESPUESTA
0,Disponibilidad de Turno,558565,2207197,84339,"Aldecoa,Juan P.",5
3,Disponibilidad de Turno,558568,3152266,21374,"Zalazar,María del Carmen",5
4,Disponibilidad de Turno,558574,2098255,67581,"Pagola,José A.",5
5,Disponibilidad de Turno,558575,2098255,103103,"Barreña,María Alejandra",5
6,Disponibilidad de Turno,558576,3445461,67581,"Pagola,José A.",5
...,...,...,...,...,...,...
99108,Atención Médica,582269,2682570,0,Sin Asignar,2
99111,Atención Médica,582714,3099663,0,Sin Asignar,2
99112,Atención Médica,582715,3099663,0,Sin Asignar,2
99113,Atención Médica,582716,3099663,0,Sin Asignar,2


In [421]:
df = df[df["id_pre_prestador"] != 0]

In [424]:
def calculador_promedios(df):
    q_respuestas = df.groupby("id_pre_prestador")["id_encuesta"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_respuestas')
    q_afiliados = df.groupby("id_pre_prestador")["id_afi_afiliado"].nunique().rename_axis('id_pre_prestador').reset_index(name='q_afiliados')
    df = df.merge(q_respuestas).merge(q_afiliados)
    pivot = pd.pivot_table(df, 
             index=["id_pre_prestador","desc_pre_nombre", "q_afiliados", "q_respuestas"], 
             columns="id", 
             values="RESPUESTA", aggfunc= ["mean", "sum"]).reset_index()
    pivot.columns = ["Prestador", 'Razón social', 'Q afiliados',
                     "Q respuestas", 'Atención Médica m', 'Disponibilidad de Turno m',
           'Limpieza y Confort m', 'Puntualidad m', 'Trato y Cordialidad m',
           'Atención Médica s', 'Disponibilidad de Turno s', 'Limpieza y Confort s',
           'Puntualidad s', 'Trato y Cordialidad s']

    pivot.rename(columns={
                'Atención Médica s':"Calificación Atención Médica",
                'Disponibilidad de Turno s':"Calificación Disponibilidad de Turno",
                'Limpieza y Confort s':"Calificación Limpieza y Confort",
                'Puntualidad s': "Calificación Puntualidad", 
                'Trato y Cordialidad s':"Calificación Trato y Cordialidad",
                'Atención Médica m':"Promedio Calificación Atención Médica", 
                'Disponibilidad de Turno m':"Promedio Calificación Disponibilidad de Turno", 
                'Limpieza y Confort m':"Promedio Calificación Limpieza y Confort",
                'Puntualidad m':"Promedio Calificación Puntualidad",
                'Trato y Cordialidad m':"Promedio Calificación Trato y Cordialidad"}, inplace=True)

    pivot["Promedio total"] = (pivot["Promedio Calificación Atención Médica"]+
                              pivot["Promedio Calificación Disponibilidad de Turno"]+
                              pivot["Promedio Calificación Limpieza y Confort"]+
                              pivot["Promedio Calificación Puntualidad"]+
                              pivot["Promedio Calificación Trato y Cordialidad"]) / 5

    pivot= pivot[['Prestador', 'Razón social', 'Q respuestas', 'Q afiliados',
                     'Calificación Atención Médica', 'Promedio Calificación Atención Médica',
                     'Calificación Disponibilidad de Turno','Promedio Calificación Disponibilidad de Turno',
                     'Calificación Limpieza y Confort',   'Promedio Calificación Limpieza y Confort',
                     'Calificación Puntualidad',   'Promedio Calificación Puntualidad',
                     'Calificación Trato y Cordialidad',  'Promedio Calificación Trato y Cordialidad','Promedio total']]
    pivot = pivot.sort_values(by="Promedio total", ascending= True)
#   pivot = pivot[pivot["Q afiliados"] >5]
    pivot = pivot.round(2)
    #promedios_totales = pd.DataFrame(pivot[['Promedio Calificación Atención Médica', 'Promedio Calificación Disponibilidad de Turno', 'Promedio Calificación Limpieza y Confort', 'Promedio Calificación Puntualidad',   'Promedio Calificación Trato y Cordialidad',  'Promedio total']].mean()).reset_index().rename(columns={0:"Promedios totales", "index": "Calificación"})
    return pivot

In [425]:
final_calif  =calculador_promedios(df)

In [426]:
metrica_calif = final_calif.sort_values(by="Q afiliados").head(100)

In [427]:
metrica_calif

,Prestador,Razón social,Q respuestas,Q afiliados,Calificación Atención Médica,Promedio Calificación Atención Médica,Calificación Disponibilidad de Turno,Promedio Calificación Disponibilidad de Turno,Calificación Limpieza y Confort,Promedio Calificación Limpieza y Confort,Calificación Puntualidad,Promedio Calificación Puntualidad,Calificación Trato y Cordialidad,Promedio Calificación Trato y Cordialidad,Promedio total
1359,124487,"Lavergata,María Laura",1,1,1,1.0,1,1.0,1,1.0,1,1.0,1,1.0,1.0
1725,193680,"Quiñones,Victor",3,1,15,5.0,15,5.0,15,5.0,15,5.0,15,5.0,5.0
1726,193685,"Galvano,Roxana",1,1,5,5.0,5,5.0,5,5.0,5,5.0,5,5.0,5.0
1429,145723,"Arvilly,Gisell M.",3,1,15,5.0,15,5.0,15,5.0,15,5.0,15,5.0,5.0
1733,195638,"Grzona,Esteban G.",1,1,5,5.0,5,5.0,5,5.0,5,5.0,5,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,26909,"Gondra,Hugo D.",1,1,5,5.0,5,5.0,5,5.0,5,5.0,5,5.0,5.0
444,27046,"Solari,Eduardo A.",1,1,5,5.0,5,5.0,5,5.0,5,5.0,5,5.0,5.0
367,25897,"Biondi,Jorge L.",1,1,5,5.0,5,5.0,5,5.0,5,5.0,5,5.0,5.0
446,27078,"Romiglio,Eduardo G.",2,1,10,5.0,10,5.0,10,5.0,10,5.0,10,5.0,5.0
